# Training and Testing an MDRNN on Different Models

This script tests the MDRNN on different models

In [ ]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from context import *
import empi_mdrnn
import tensorflow as tf
from keras import backend as K

In [ ]:
# Model Hyperparameters
mdrnn_units = 32
mdrnn_mixes = 5
mdrnn_layers = 2

pitemp = 1
sigmatemp = 0.01

empi_mdrnn.MODEL_DIR = "./models/"
net = empi_mdrnn.PredictiveMusicMDRNN(mode=empi_mdrnn.NET_MODE_RUN,
                                              dimension=2,
                                              n_hidden_units=mdrnn_units,
                                              n_mixtures=mdrnn_mixes,
                                              layers=mdrnn_layers)
net.pi_temp = pitemp
net.sigma_temp = sigmatemp

In [ ]:
# test untrained model.
#net.model.load_weights("../models/musicMDRNN-dim2-layers2-units32-mixtures5-scale10-human.h5")
net.prepare_model_for_running()
sample = empi_mdrnn.random_sample()
num_steps = 1500
p = net.generate_performance(sample, num_steps)
perf_df = pd.DataFrame({'t':p.T[0], 'x':p.T[1]})
perf_df['time'] = perf_df.t.cumsum()
#plt.plot(perf_df.time, perf_df.x, '.r-')
#plt.show()
display(perf_df.describe())

# Generate slightly nicer plot.
ax = perf_df.plot(x='time', y='x', kind="line", color='g', figsize=(15,4), legend=False)
ax.set_xlabel("seconds")
ax.set_ylabel("position")
ax.get_figure().savefig('../images/untrained_data_output.pdf', dpi=300, bbox_inches="tight")
ax.get_figure().savefig('../images/untrained_data_output.png', dpi=300, bbox_inches="tight")

In [ ]:
# Load and test synthetic model.
net.model.load_weights("../models/musicMDRNN-dim2-layers2-units32-mixtures5-scale10-synth.h5")
net.prepare_model_for_running()
sample = empi_mdrnn.random_sample()
num_steps = 1500
p = net.generate_performance(sample, num_steps)
perf_df = pd.DataFrame({'t':p.T[0], 'x':p.T[1]})
perf_df['time'] = perf_df.t.cumsum()
display(perf_df.describe())

# Generate slightly nicer plot.
ax = perf_df.plot(x='time', y='x', kind="line", color='r', figsize=(15,4), legend=False)
ax.set_xlabel("seconds")
ax.set_ylabel("position")
ax.get_figure().savefig('../images/synth_data_output.pdf', dpi=300, bbox_inches="tight")
ax.get_figure().savefig('../images/synth_data_output.png', dpi=300, bbox_inches="tight")

In [ ]:
# Load and test synthetic model.
net.model.load_weights("../models/musicMDRNN-dim2-layers2-units32-mixtures5-scale10-human.h5")
net.prepare_model_for_running()
sample = empi_mdrnn.random_sample()
num_steps = 1500
p = net.generate_performance(sample, num_steps)
perf_df = pd.DataFrame({'t':p.T[0], 'x':p.T[1]})
perf_df['time'] = perf_df.t.cumsum()
#plt.plot(perf_df.time, perf_df.x, '.r-')
#plt.show()
display(perf_df.describe())

# Generate slightly nicer plot.
ax = perf_df.plot(x='time', y='x', kind="line", color='b', figsize=(15,4), legend=False)
ax.set_xlabel("seconds")
ax.set_ylabel("position")
ax.get_figure().savefig('../images/human_data_output.pdf', dpi=300, bbox_inches="tight")
ax.get_figure().savefig('../images/human_data_output.png', dpi=300, bbox_inches="tight")

In [ ]:
# ## Investigate Output
# window = 100
# for n in [1000,2000,3000,4000,5000,6000]:
#     print("Window:", str(n),'to',str(n+window))
#     plt.plot(perf_df[n:n+window].time, perf_df[n:n+window].x, '.r-')
#     plt.show()